In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [2]:
tf.__version__

'2.7.0'

In [3]:
df_ = pd.read_csv('Churn_Modelling.csv')
X = df_.iloc[:, 3:-1].values
y = df_.iloc[:,-1].values

### Data Processing

In [4]:
# Label encode Gender column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, -2])

In [5]:
# OneHot encode region
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [6]:
X.shape

(10000, 12)

In [7]:
# Train test split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# Feature scaling

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

### Building the ANN

In [9]:
ann = tf.keras.models.Sequential()

# Add input layer and first hidden layer

# 6 is sort of random
# relu = rectifier; this is what it has to be for a fully connected neural network 
ann.add(tf.keras.layers.Dense(units=6, activation='relu')) 

# second hidden layer
ann.add(tf.keras.layers.Dense(units=6, activation='relu')) 

# add output layer
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid')) # would be sigmoid if it was categorical

### Train the ANN

* Good article on binary cross entropy: https://towardsdatascience.com/understanding-binary-cross-entropy-log-loss-a-visual-explanation-a3ac6025181a

In [14]:
y_train

array([0, 0, 1, ..., 1, 1, 0], dtype=int64)

In [65]:
# Compile the ANN
ann.compile(
    optimizer='adam', #  very performant optimizer that performs stochastic gradient descent; this 
                      # updates the weights 
    loss='binary_crossentropy',  # could be category_crossentropy for categorical;
                                # computes the difference between prediction and real
    metrics=['accuracy']
)

# Train the ANN on the training set

ann.fit(
    x=X_train,
    y=y_train,
    batch_size= 32,  # instead of comparing prediction to real one by one, do it 32 at a time
    epochs = 100
)

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.5796 - accuracy: 0.7471
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4808 - accuracy: 0.8064
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4469 - accuracy: 0.8140
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4328 - accuracy: 0.8190
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4222 - accuracy: 0.8248
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4110 - accuracy: 0.8317
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4016 - accuracy: 0.8369
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3915 - accuracy: 0.8424
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3832 - accuracy: 0.8462
Epoch 10/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3757 - accura

###  Predict using ANN

In [71]:
# Predicting one very specific person
ann.predict(sc.transform(
    [[
        1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000
    ]]
    )
)

array([[0.04535422]], dtype=float32)

In [75]:
y_hat.reshape(len(y_hat), 1)

array([[False],
       [False],
       [False],
       ...,
       [ True],
       [False],
       [False]])

In [76]:
# Predict the test set 

y_hat = ann.predict(X_test)
y_hat = (y_hat > 0.5)
np.concatenate((
    y_hat.reshape(len(y_hat), 1), y_test.reshape(len(y_test), 1)),1)

array([[0, 0],
       [0, 0],
       [0, 0],
       ...,
       [1, 1],
       [0, 1],
       [0, 1]], dtype=int64)

In [80]:
# Confusion matrix
from sklearn.metrics import confusion_matrix, accuracy_score 
cm = confusion_matrix(y_test, y_hat)
print(cm)
accuracy_score(y_test, y_hat)

[[1532   75]
 [ 203  190]]


0.861